## Expected Results:
In this data analysis project, having 3 categories to be potentially used for the prediction of a patient's stroke status, we aim to find the best knn model between them. with the 3 categories being demographics, health, and lifestyle, we predict that the health category involving hypertension, heart disease and stress level predictors will have the highest accuracy to be used as a prediction model to find out a patient's stroke history status. 

## We do this because ...
The findings of the data study will benefit clinical, public and personal health. Knowing the most impactful factors, healthcare professionals can determine if someone is at a higher risk of having a stroke by putting more focus on those factors. Depending on the outcome of our study, people can improve their lifestyle habits and take preventive measures to reduce their risks of getting a stroke.

## But what about...
Future questions:
* Are there other demographic, health, or lifestyle factors that were not considered in this study but might also influence stroke risk?
* Can predictive models be developed to estimate an individual's stroke risk more accurately based on multiple risk factors?
* What are the long-term effects of interventions aimed at reducing stroke risk?

<img src="https://media.giphy.com/media/3orieTTZVPaVNMhRra/giphy.gif" width = "600"/>

<img src="https://vitals.sutterhealth.org/files/2020/05/iStock-1168179082.jpg" width = "600"/>

## Sources
* Dataset: https://www.kaggle.com/datasets/teamincribo/stroke-prediction/data
* Gif: https://media.giphy.com/media/3orieTTZVPaVNMhRra/giphy.gif
* Stroke Image: https://vitals.sutterhealth.org/files/2020/05/iStock-1168179082.jpg
* Stroke background information: https://www.cdc.gov/stroke/about.htm#:~:text=A%20stroke%2C%20sometimes%20called%20a,term%20disability%2C%20or%20even%20death.

In [1]:
#### This is the main data file (to be handed in)#####

# Please read the following before adding changes to this document:
# 1. Does the code you are running work? (Have you run the code in your personal draft file and it returns no error?)
# 2. Have you consulted in the group chat before committing your code?
# 3. Be sure to add a comment for what the code added is for, and also your initials, so we can track changes!
# 4. Commit often so that we can see the history of changes.
# 5. Before pushing your changes to GitHub, remember that all changes will override the final file.

# Example below - Ching has added the data file.